# Build feature matrices
> 1. Load raw_partD.csv

In [1]:
import pandas as pd
import src.scripts as src
import numpy as np

In [2]:
partD = src.read_from_efs('raw_PartD.csv')

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
features = ['bene_count', 
            'total_claim_count',
            'total_30_day_fill_count',
            'total_day_supply',
            'total_drug_cost',
            'bene_count_ge65',
            'total_claim_count_ge65',
            'total_30_day_fill_count_ge65',
            'total_day_supply_ge65',
            'total_drug_cost_ge65']

drop_features = ['description_flag', 
                 'drug_name',
                 'bene_count_ge65_suppress_flag',
                 'ge65_suppress_flag']

id_features = ['year',
               'npi',
               'nppes_provider_last_org_name',
               'nppes_provider_first_name',
               'nppes_provider_city',
               'nppes_provider_state',
               'specialty_description']

In [4]:
partD1 = partD.drop(labels=drop_features, axis=1).reset_index(drop=True)
partD1.head()

,npi,nppes_provider_last_org_name,nppes_provider_first_name,nppes_provider_city,nppes_provider_state,specialty_description,generic_name,bene_count,total_claim_count,total_30_day_fill_count,total_day_supply,total_drug_cost,bene_count_ge65,total_claim_count_ge65,total_30_day_fill_count_ge65,total_day_supply_ge65,total_drug_cost_ge65,year
0,1003000126,ENKESHAFI,ARDALAN,CUMBERLAND,MD,Internal Medicine,ISOSORBIDE MONONITRATE,NaN,11,11.0,307,171.59,NaN,NaN,NaN,NaN,NaN,2013
1,1003000126,ENKESHAFI,ARDALAN,CUMBERLAND,MD,Internal Medicine,LEVOFLOXACIN,26.0,26,26.0,165,227.10,15.0,15.0,15.0,106.0,159.72,2013
2,1003000126,ENKESHAFI,ARDALAN,CUMBERLAND,MD,Internal Medicine,LISINOPRIL,17.0,19,19.0,570,100.37,NaN,NaN,NaN,NaN,NaN,2013
3,1003000126,ENKESHAFI,ARDALAN,CUMBERLAND,MD,Internal Medicine,METOPROLOL TARTRATE,28.0,30,31.0,916,154.65,NaN,NaN,NaN,NaN,NaN,2013
4,1003000126,ENKESHAFI,ARDALAN,CUMBERLAND,MD,Internal Medicine,PREDNISONE,14.0,14,14.0,133,44.72,NaN,NaN,NaN,NaN,NaN,2013


In [17]:
df = partD1.groupby(['year', 'npi', 'nppes_provider_last_org_name', \
                     'nppes_provider_first_name', 'specialty_description', \
                     'nppes_provider_city', 'nppes_provider_state','generic_name']).agg('sum').reset_index()

In [18]:
list(df)

['year',
 'npi',
 'nppes_provider_last_org_name',
 'nppes_provider_first_name',
 'specialty_description',
 'nppes_provider_city',
 'nppes_provider_state',
 'generic_name',
 'bene_count',
 'total_claim_count',
 'total_30_day_fill_count',
 'total_day_supply',
 'total_drug_cost',
 'bene_count_ge65',
 'total_claim_count_ge65',
 'total_30_day_fill_count_ge65',
 'total_day_supply_ge65',
 'total_drug_cost_ge65']

In [19]:
df_id = df[['year', 'npi', 'nppes_provider_last_org_name', 'nppes_provider_first_name', 'specialty_description', \
            'nppes_provider_city', 'nppes_provider_state']].copy()

df.drop(['year', 'npi', 'nppes_provider_last_org_name', 'nppes_provider_first_name', 'specialty_description', \
         'nppes_provider_city', 'nppes_provider_state'], axis=1, inplace=True)

print('df_id shape: {}, df shape: {}'.format(df_id.shape, df.shape))

df_id shape: (90624128, 7), df shape: (90624128, 11)


In [ ]:
src.save_to_efs(df_id, 'df_id.csv')
del df_id, partD, partD1

In [27]:
generic_names_df = df['generic_name']
df.drop('generic_name',axis=1, inplace=True)
print('generic_names_df shape: {}, df shape: {}'.format(generic_names_df.shape, df.shape))

generic_names_df shape: (90624128,), df shape: (90624128, 10)


In [29]:
df['total_drug_cost_per_beneficiary'] = df['total_drug_cost']/df['bene_count']
df['total_drug_cost_per_claim'] = df['total_drug_cost']/df['total_claim_count']
df['total_drug_cost_per_beneficiary_ge65'] = df['total_drug_cost_ge65']/df['bene_count_ge65']
df['total_drug_cost_per_claim_ge65'] = df['total_drug_cost_ge65']/df['total_claim_count_ge65']
print('features matrix shape with `implicit features` included: ', df.shape)

features matrix shape with `implicit features` included:  (90624128, 14)


In [30]:
src.save_to_efs(df, 'df.csv')

In [33]:
dummies_df = pd.get_dummies(generic_names_df, sparse=True)
print('`generic_name` dummies shape: ', dummies_df.shape)

`generic_name` dummies shape:  (90624128, 2000)


In [37]:
print('`generic_name` dummies size in MiB: ', sum(dummies_df.memory_usage())/10**6)


`generic_name` dummies size in MiB:  90.624208


In [38]:
df1 = pd.concat([df, dummies_df], axis=1)
print('Shape after concatinating `explicit features` and `generic_name` dumies: ', df1.shape)

Shape after concatinating `explicit features` and `generic_name` dumies:  (90624128, 2014)


In [40]:
type(df1.values)
# np.savetxt("X.csv", a, delimiter=",")

MemoryError: 